# Semantic Kernel with OpenBnB MCP Server Integration

यो नोटबुकले Semantic Kernel लाई वास्तविक OpenBnB MCP सर्भर संग कसरी प्रयोग गर्ने देखाउँछ ताकि MCPStdioPlugin प्रयोग गरी वास्तविक Airbnb आवासहरूको खोजी गर्न सकियोस्। LLM पहुँचका लागि, यसले Microsoft Foundry को प्रयोग गर्छ। तपाईंले आफ्नो वातावरण चरहरू सेटअप गर्न, तपाईंले [Setup Lesson ](/00-course-setup/README.md) अनुसरण गर्न सक्नुहुन्छ।


## आवश्यक प्याकेजहरू आयात गर्नुहोस्


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP प्लगइन कनेक्शन सिर्जना गर्दै

हामी MCPStdioPlugin प्रयोग गरी [OpenBnB MCP सर्भर](https://github.com/openbnb-org/mcp-server-airbnb) मा कनेक्ट गर्नेछौं। यो सर्भर @openbnb/mcp-server-airbnb प्याकेजमार्फत Airbnb खोज कार्यक्षमता प्रदान गर्दछ।


## क्लाइन्ट सिर्जना गर्दै

यस नमूनामा, हामी हाम्रो LLM पहुँचका लागि Microsoft Foundry प्रयोग गर्नेछौं। सुनिश्चित गर्नुहोस् कि तपाईंको वातावरण चरहरू ठीकसँग सेटअप छन्।


## Environment Configuration

Azure OpenAI सेटिङहरू कन्फिगर गर्नुहोस्। सुनिश्चित गर्नुहोस् कि तपाइँसँग निम्न वातावरण चरहरू सेट गरिएको छ:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP एकीकरण बुझ्दै

यो नोटबुकले **वास्तविक OpenBnB MCP सर्भर** सँग जडान गर्छ जसले वास्तविक Airbnb खोज सुविधा प्रदान गर्दछ।

### यसले कसरी काम गर्छ:

1. **MCPStdioPlugin**: MCP सर्भरसँग मानक इनपुट/आउटपुट सञ्चार प्रयोग गर्छ
2. **वास्तविक NPM प्याकेज**: npx मार्फत `@openbnb/mcp-server-airbnb` डाउनलोड र चलाउँछ
3. **प्रत्यक्ष डेटा**: तिनीहरूको API बाट वास्तविक Airbnb सम्पत्ती डेटा फर्काउँछ
4. **कार्य खोजी**: एजेन्टले स्वचालित रूपमा MCP सर्भरबाट उपलब्ध कार्यहरू पत्ता लगाउँछ

### उपलब्ध कार्यहरू:

OpenBnB MCP सर्भरले सामान्यतया प्रदान गर्दछ:
- **search_listings** - स्थान र मापदण्ड अनुसार Airbnb सम्पत्तीहरूको खोजी गर्नुहोस्
- **get_listing_details** - निर्दिष्ट सम्पत्तीहरूको विस्तृत जानकारी प्राप्त गर्नुहोस्
- **check_availability** - निर्दिष्ट मितिहरूको लागि उपलब्धता जाँच गर्नुहोस्
- **get_reviews** - सम्पत्तीहरूको लागि समीक्षा प्राप्त गर्नुहोस्
- **get_host_info** - सम्पत्ती होस्टहरूको जानकारी प्राप्त गर्नुहोस्

### पूर्वआवश्यकताहरू:

- तपाईंको सिस्टममा **Node.js** स्थापना गरिएको हुनुपर्छ
- MCP सर्भर प्याकेज डाउनलोड गर्न **इन्टरनेट कनेक्शन** आवश्यक छ
- **NPX** उपलब्ध हुनुपर्छ (Node.js सँगै आउँछ)

### जडान परीक्षण:

तपाईं म्यानुअली MCP सर्भर चलाएर परीक्षण गर्न सक्नुहुन्छ:
```bash
npx -y @openbnb/mcp-server-airbnb
```

यसले OpenBnB MCP सर्भर डाउनलोड र सुरू गर्छ, जुन पछि Semantic Kernel ले वास्तविक Airbnb डेटा प्राप्त गर्न जडान गर्छ।


## OpenBnB MCP सर्भरसँग एजेन्ट चलाउँदै

अब हामी AI एजेन्ट चलाउनेछौं जसले OpenBnB MCP सर्भरसँग जडान गरी स्टकहोल्ममा २ वयस्क र १ बच्चाका लागि वास्तविक Airbnb आवासहरू खोज्नेछ। खोज मापदण्ड परिवर्तन गर्न `user_inputs` सूचीलाई स्वतन्त्र रूपमा परिवर्तन गर्न सक्नुहुन्छ।


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# सारांश
बधाई छ! तपाईंले सफलतापूर्वक एउटा AI एजेन्ट निर्माण गर्नुभएको छ जसले Model Context Protocol (MCP) प्रयोग गरी वास्तविक संसारको आवास खोजसँग एकीकृत गर्दछ:

## प्रयोग गरिएको प्रविधिहरू:
- Semantic Kernel - Azure OpenAI सँग बुद्धिमान एजेन्टहरू निर्माण गर्नका लागि
- Microsoft Foundry - LLM क्षमताहरू र च्याट पूर्णताको लागि
- MCP (Model Context Protocol) - मानकीकृत उपकरण एकीकरणका लागि
- OpenBnB MCP Server - वास्तविक Airbnb खोज कार्यक्षमताका लागि
- Node.js/NPX - बाह्य MCP सर्भर चलाउनका लागि

## तपाईंले के सिक्नुभयो:
- MCP एकीकरण: Semantic Kernel एजेन्टहरूलाई बाह्य MCP सर्भरहरूसँग जडान गर्ने
- वास्तविक-समय डाटा पहुँच: प्रत्यक्ष APIs मार्फत वास्तविक Airbnb सम्पत्तिहरू खोज्ने
- प्रोटोकल सञ्चार: एजेन्ट र MCP सर्भरबीच stdio सञ्चार प्रयोग गर्ने
- कार्य फेला पार्ने: MCP सर्भरहरूसँग उपलब्ध कार्यहरू स्वचालित रूपमा पत्ता लगाउने
- स्ट्रिमिङ प्रतिक्रिया: कार्य कलहरू वास्तविक-समयमा कैद र लग गर्ने
- HTML रेंडरिङ: एजेन्ट प्रतिक्रिया स्टाइल गरिएको तालिका र अन्तरक्रियात्मक प्रदर्शनहरूसँग स्वरूप गर्ने

## आगामी कदमहरू:
- थप MCP सर्भरहरू एकीकृत गर्ने (मौसम, उडानहरू, रेस्टुराँटहरू)
- MCP र A2A प्रोटोकलहरू संयोजन गरी बहु-एजेन्ट प्रणाली निर्माण गर्ने
- आफ्नै डाटा स्रोतहरूको लागि कस्टम MCP सर्भरहरू सिर्जना गर्ने
- सत्रहरूमा दिगो संवाद स्मृति कार्यान्वयन गर्ने
- MCP सर्भर संयोजनसहित एजेन्टलाई Azure Functions मा परिनियोजन गर्ने
- प्रयोगकर्ता प्रमाणीकरण र बुकिंग क्षमताहरू थप्ने


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) को प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताका लागि प्रयास गर्दछौं, तर कृपया जानकार हुनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धता हुन सक्छ। मूल दस्तावेज़ यसको मूल भाषामा नै अधिकारिक स्रोत मानिनु पर्छ। महत्वपूर्ण जानकारीका लागि व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
